In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from tensorflow.python.keras import models
from tensorflow.python.keras import layers

In [149]:
allDeaths = pd.read_csv("datasets/all_deaths.csv", encoding='latin-1')
allJails = pd.read_csv("datasets/all_jails.csv", encoding='latin-1')

# https://bjs.ojp.gov/library/publications/prisoners-2021-statistical-tables#:~:text=The%20U.S.%20prison%20population%20was,decrease%20from%202011%20(1%2C599%2C000)
prisonerDemographics = pd.read_csv("datasets/p21stt01.csv", encoding='latin-1')
prisonersPerYear = pd.read_csv("datasets/p21stat02.csv", encoding='latin-1')

# https://www.kaggle.com/datasets/christophercorrea/prisoners-and-crime-in-united-states?select=crime_and_incarceration_by_state.csv
crimesAndIncarcerationsByState = pd.read_csv("datasets/crime_and_incarceration_by_state.csv", encoding='latin-1')
prisonCustodyByState = pd.read_csv("datasets/prison_custody_by_state.csv", encoding='latin-1')
crimeTotalsByState = pd.read_csv("datasets/ucr_by_state.csv", encoding='latin-1')


In [150]:
# clean jail death data

# allDeaths data frame -> might not be too useful
allDeaths = allDeaths.drop(["jail", "id", "full_name", "last_name", "first_name", "mid_name", "race_detail", "what_info_from_media", "cause_detail"], axis=1)

# allJails data frame
allJails = allJails.drop(["id", "fips", "state_notes", "jail_notes"], axis = 1)


In [151]:
# clean prisoner data

# prisonerDemographics data frame
prisonerDemographics = prisonerDemographics.drop(range(0, 10)).reset_index(drop=True)
prisonerDemographics = prisonerDemographics.drop(range(11, 17)).reset_index(drop=True)
prisonerDemographics = prisonerDemographics.drop(["Unnamed: 1", "Unnamed: 3", "Unnamed: 5", "Unnamed: 7", "Unnamed: 9", "Unnamed: 11"], axis=1)
prisonerDemographics.columns = ["Year", "Total", "Federal", "State", "Male", "Female"]

# prisonersPerYear data frame
prisonersPerYear = prisonersPerYear.drop(range(0, 10)).reset_index(drop=True)
prisonersPerYear = prisonersPerYear.drop(range(26, 28)).reset_index(drop=True)
prisonersPerYear.columns = ["Year", "Sentenced Prisoners"]


In [152]:
# clean crime data

crimeTotalsByState = crimeTotalsByState.drop(["Unnamed: 15", "Unnamed: 16", "Unnamed: 17", "Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "crime_reporting_change", "crimes_estimated"], axis=1)
crimeTotalsByState = crimeTotalsByState.drop(range(869, 948)).reset_index(drop=True)
crimeTotalsByState = crimeTotalsByState.rename(columns={"ï»¿jurisdiction": "State"})
crimeTotalsByState["year"] = crimeTotalsByState["year"].astype(int)
 
crimeTotalsByState


,State,year,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
0,Alaska,2001,"633,630","3,735",39,501,NaN,514,"2,681","23,160","3,847","16,695","2,618"
1,Alaska,2002,"641,482","3,627",33,511,NaN,489,"2,594","24,118","3,908","17,739","2,471"
2,Alaska,2003,"648,280","3,877",39,605,NaN,446,"2,787","24,386","3,874","18,051","2,461"
3,Alaska,2004,"657,755","4,159",37,558,NaN,447,"3,117","22,172","3,773","16,159","2,240"
4,Alaska,2005,"663,253","4,194",32,538,NaN,537,"3,087","23,975","4,131","17,249","2,595"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
864,Oregon,2017,4142776,11674,104,NaN,1999,2432,7139,123722,17705,88877,17140
865,Washington,2016,7280934,22101,195,NaN,3133,5649,13124,254994,49249,173423,32322
866,Washington,2017,7405743,22548,230,NaN,3255,5390,13673,235027,43720,162511,28796
867,Puerto Rico,2016,3406520,7643,679,NaN,169,3201,3594,35201,8251,23163,3787
